In [1]:
%pip install --upgrade --quiet pip setuptools wheel
%pip install --upgrade --quiet  langchain langchain-openai faiss-cpu tiktoken crate 'crate[sqlalchemy]' pandas jq 
%pip install --use-pep517 --quiet python-dotenv

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


# Use Mistral-7B as LLM and sentence_transformers for embeddings, FAISS as retriever

## Setup environment variables

In [2]:
import os

from dotenv import load_dotenv

load_dotenv()

True

## RAG search, indexing pipeline

In [3]:

from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.retrievers import BaseRetriever
from langchain_core.callbacks import CallbackManagerForRetrieverRun
from langchain_core.documents import Document
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import JSONLoader, DirectoryLoader
from typing import List

In [4]:
# Define the metadata extraction function.
def metadata_func(record: dict, metadata: dict) -> dict:
    metadata["source_url"] = record.get("url")
    metadata["source_title"] = record.get("title")

    if "source" in metadata:
        metadata["source"] = metadata["source_url"]

    return metadata


loader = DirectoryLoader(
    './',
    glob="everything-*.json",

    loader_cls=JSONLoader,
    loader_kwargs={
        "jq_schema": ".[]",
        "text_content": False,
        "content_key": "html",
        "metadata_func": metadata_func,
    }
)

data = loader.load()
data[:2]

[Document(page_content="This website stores cookies on your computer. These cookies are used to collect information about how you interact with our website and allow us to remember you. We use this information in order to improve and customize your browsing experience and for analytics and metrics about our visitors both on this website and other media. To find out more about the cookies we use, see our Privacy Policy\n\nIf you decline, your information won’t be tracked when you visit this website. A single cookie will be used in your browser to remember your preference not to be tracked.\n\nSettings\nAccept\nDecline\n\nThe Guide for Time Series Data Projects is out.\n\n Download now\nSkip to content\nProduct Solutions Customers Resources Documentation\nLog In\nGet Started\nCompany\nBlog\nALL\n \nPRODUCT\n \nGENERAL\n \nOPERATIONS\n \nDEVELOPMENT\n \nCOMMUNITY\n \nCOMPANY\n \nINTEGRATIONS\n \nNEWSLETTER\nGENERAL PHP\nHow the Fastly Wordpress Plugin Helped Us Deal with a Massive Traffic

In [5]:
text_splitter = RecursiveCharacterTextSplitter(
    separators=[
        "\n\n",
        "\n",
        " ",
        ".",
        ",",
    ],
    chunk_size=500,
    chunk_overlap=50,
    length_function=len,
    is_separator_regex=False,
)

docs_splits = text_splitter.split_documents(data)
docs_splits[:2]


[Document(page_content='This website stores cookies on your computer. These cookies are used to collect information about how you interact with our website and allow us to remember you. We use this information in order to improve and customize your browsing experience and for analytics and metrics about our visitors both on this website and other media. To find out more about the cookies we use, see our Privacy Policy', metadata={'source': 'https://cratedb.com/blog/page/30', 'seq_num': 1, 'source_url': 'https://cratedb.com/blog/page/30', 'source_title': 'CrateDB Blog | Development, integrations, IoT, & more (30)'}),
 Document(page_content='If you decline, your information won’t be tracked when you visit this website. A single cookie will be used in your browser to remember your preference not to be tracked.\n\nSettings\nAccept\nDecline\n\nThe Guide for Time Series Data Projects is out.', metadata={'source': 'https://cratedb.com/blog/page/30', 'seq_num': 1, 'source_url': 'https://crated

## Setup LLamaCPP with Metal

In [6]:
# %pip install --upgrade --quiet  llama-cpp-python

In [7]:
!CMAKE_ARGS="-DLLAMA_METAL=on" FORCE_CMAKE=1 pip install llama-cpp-python

## Download model

In [8]:
# !huggingface-cli download TheBloke/Mistral-7B-v0.1-GGUF mistral-7b-v0.1.Q4_K_M.gguf --local-dir downloads --local-dir-use-symlinks False

In [9]:
from langchain_community.llms import LlamaCpp
from langchain_core.callbacks import CallbackManager, StreamingStdOutCallbackHandler
from langchain_core.prompts import PromptTemplate

In [10]:
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])

# Make sure the model path is correct for your system!
llm = LlamaCpp(
    model_path="downloads/mistral-7b-v0.1.Q4_K_M.gguf",
    temperature=0.75,
    max_tokens=2000,
    top_p=1,
    callback_manager=callback_manager,
    verbose=True,  # Verbose is required to pass to the callback manager
)

llama_model_loader: loaded meta data with 20 key-value pairs and 291 tensors from downloads/mistral-7b-v0.1.Q4_K_M.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = mistralai_mistral-7b-v0.1
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:                 llama.attention.he

In [11]:
llm.invoke("How to limit permissions?")

I have a user that needs access to a specific folder in one of my workspaces. This is not an administrator account, so I do not want to give it administrative rights on the computer. If I create a local group and add this user to the group then grant the group “full control” rights on the folder, does this allow access? Will that also allow the user to see all other files on the computer?
Thanks!
Hi, you can’t give only specific folder permissions without admin account.
I recommend you create a new account with administrator privileges and grant this account full control of the required folder and then assign this account to the users group that is not administrator and the user will be able to access the files in this folder.
Please let us know if you have any other question.
Hi, thanks for the reply. If I create a local user, grant it full control of the required folder but then add the user to the group “Users”, will it allow access?


llama_print_timings:        load time =     292.98 ms
llama_print_timings:      sample time =      19.88 ms /   215 runs   (    0.09 ms per token, 10813.80 tokens per second)
llama_print_timings: prompt eval time =     292.91 ms /     6 tokens (   48.82 ms per token,    20.48 tokens per second)
llama_print_timings:        eval time =   11395.13 ms /   214 runs   (   53.25 ms per token,    18.78 tokens per second)
llama_print_timings:       total time =   12077.08 ms /   220 tokens


'\n\nI have a user that needs access to a specific folder in one of my workspaces. This is not an administrator account, so I do not want to give it administrative rights on the computer. If I create a local group and add this user to the group then grant the group “full control” rights on the folder, does this allow access? Will that also allow the user to see all other files on the computer?\n\nThanks!\n\nHi, you can’t give only specific folder permissions without admin account.\nI recommend you create a new account with administrator privileges and grant this account full control of the required folder and then assign this account to the users group that is not administrator and the user will be able to access the files in this folder.\n\nPlease let us know if you have any other question.\n\nHi, thanks for the reply. If I create a local user, grant it full control of the required folder but then add the user to the group “Users”, will it allow access?'

# Setup embedding model

In [12]:
%pip install --upgrade --quiet  sentence_transformers > /dev/null

Note: you may need to restart the kernel to use updated packages.


In [13]:
from langchain_community.embeddings import HuggingFaceEmbeddings

In [14]:
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

In [15]:
# Indexing in FAISS

db = FAISS.from_documents(docs_splits, embeddings)

In [43]:
retriever = db.as_retriever(
    # search_type="mmr",
    search_kwargs={'k': 3, 'fetch_k': 100}
)
retriever

VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x5acc1a950>, search_kwargs={'k': 3, 'fetch_k': 100})

In [44]:
import json

In [45]:
template = """Answer the question based only on the following context, if possible use links inside answer to reference the source, use markdown:

today date is 2024 April 3rd

{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
# model = ChatOpenAI()


def format_docs(docs):
    breakpoint()
    return json.dumps([{"text": d.page_content, "source": d.metadata.get('source')} for d in docs])


chain = (
        {"context": retriever | format_docs,
         "question": RunnablePassthrough()}
        | prompt
        | llm
        | StrOutputParser()
)

# result = chain.invoke("How to limit permissions?")
# result = chain.invoke(" How AWS marketplace works, and why I cannot see deployment in my account?")
# result = chain.invoke("What are edge regions and how to use them?")
result = chain.invoke("Write me example of using blobs?")
# result = chain.invoke("How to use BLOB store in CrateDB? and what are the benefits?")
result


Llama.generate: prefix-match hit


Answer: https://cratedb.com/blog/using-crate-as-a-blobstore
3. Listing & Querying BLOBs\n\nTo list all blobs inside a blob table a SELECT statement can be used:\n\nsh$ curl -isSX GET '127.0.0.1:4200/_blobs/myblobs' HTTP/1.1 200 OK Content-


llama_print_timings:        load time =     292.98 ms
llama_print_timings:      sample time =      10.06 ms /   103 runs   (    0.10 ms per token, 10234.50 tokens per second)
llama_print_timings: prompt eval time =   14126.76 ms /   408 tokens (   34.62 ms per token,    28.88 tokens per second)
llama_print_timings:        eval time =    4913.73 ms /   102 runs   (   48.17 ms per token,    20.76 tokens per second)
llama_print_timings:       total time =   19266.88 ms /   510 tokens


"\nAnswer: https://cratedb.com/blog/using-crate-as-a-blobstore\n\n3. Listing & Querying BLOBs\\n\\nTo list all blobs inside a blob table a SELECT statement can be used:\\n\\nsh$ curl -isSX GET '127.0.0.1:4200/_blobs/myblobs' HTTP/1.1 200 OK Content-"

In [46]:
from IPython.display import display, Markdown

display(Markdown(result))


Answer: https://cratedb.com/blog/using-crate-as-a-blobstore

3. Listing & Querying BLOBs\n\nTo list all blobs inside a blob table a SELECT statement can be used:\n\nsh$ curl -isSX GET '127.0.0.1:4200/_blobs/myblobs' HTTP/1.1 200 OK Content-

In [47]:
display(Markdown(chain.invoke("What are edge regions and how to use them?")))

Llama.generate: prefix-match hit


Answer: An edge region is a location where CrateDB Cloud stores data in an on-premise environment. In addition, edge regions can be used by organizations that want to keep their data on-site. The edge regions command can be


llama_print_timings:        load time =     292.98 ms
llama_print_timings:      sample time =       4.71 ms /    52 runs   (    0.09 ms per token, 11030.97 tokens per second)
llama_print_timings: prompt eval time =   14268.93 ms /   410 tokens (   34.80 ms per token,    28.73 tokens per second)
llama_print_timings:        eval time =    2591.21 ms /    51 runs   (   50.81 ms per token,    19.68 tokens per second)
llama_print_timings:       total time =   17002.44 ms /   461 tokens



Answer: An edge region is a location where CrateDB Cloud stores data in an on-premise environment. In addition, edge regions can be used by organizations that want to keep their data on-site. The edge regions command can be

In [48]:
display(Markdown(chain.invoke("How AWS marketplace works, and why I cannot see deployment in my account?")))

ValueError: Requested tokens (540) exceed context window of 512

In [49]:
display(Markdown(chain.invoke("What are recent blog posts about CrateDB?")))

Llama.generate: prefix-match hit


Answer:
Hi there,
We are bringing you the latest news around CrateDB, including business news, product updates, tutorials, blog posts, and upcoming events. If you have an interesting project or use case with CrateDB that you would like to share with our community and us, just answer to this email and tell us about it.
BUSINESS NEWS
Unlocking data insights at scale for the mining industry
The mining sector is one of the most challenging industries in


llama_print_timings:        load time =     292.98 ms
llama_print_timings:      sample time =      10.74 ms /   108 runs   (    0.10 ms per token, 10055.87 tokens per second)
llama_print_timings: prompt eval time =   12184.96 ms /   354 tokens (   34.42 ms per token,    29.05 tokens per second)
llama_print_timings:        eval time =    5199.40 ms /   107 runs   (   48.59 ms per token,    20.58 tokens per second)
llama_print_timings:       total time =   17612.57 ms /   461 tokens



Answer:

Hi there,

We are bringing you the latest news around CrateDB, including business news, product updates, tutorials, blog posts, and upcoming events. If you have an interesting project or use case with CrateDB that you would like to share with our community and us, just answer to this email and tell us about it.

BUSINESS NEWS

Unlocking data insights at scale for the mining industry

The mining sector is one of the most challenging industries in

In [50]:
display(Markdown(chain.invoke("Write me example python code to use CrateDB?")))

Llama.generate: prefix-match hit


Answer: https://github.com/crate/crate/blob/master/python/crate_driver/examples/write_bulk.py


llama_print_timings:        load time =     292.98 ms
llama_print_timings:      sample time =       3.49 ms /    34 runs   (    0.10 ms per token,  9733.75 tokens per second)
llama_print_timings: prompt eval time =    7253.46 ms /   210 tokens (   34.54 ms per token,    28.95 tokens per second)
llama_print_timings:        eval time =    1671.66 ms /    33 runs   (   50.66 ms per token,    19.74 tokens per second)
llama_print_timings:       total time =    9015.15 ms /   243 tokens



Answer: https://github.com/crate/crate/blob/master/python/crate_driver/examples/write_bulk.py

In [51]:
display(Markdown(chain.invoke("Write me example golang code to use CrateDB?")))

Llama.generate: prefix-match hit


Answer:


llama_print_timings:        load time =     292.98 ms
llama_print_timings:      sample time =       0.48 ms /     5 runs   (    0.10 ms per token, 10373.44 tokens per second)
llama_print_timings: prompt eval time =   16419.25 ms /   447 tokens (   36.73 ms per token,    27.22 tokens per second)
llama_print_timings:        eval time =     199.28 ms /     4 runs   (   49.82 ms per token,    20.07 tokens per second)
llama_print_timings:       total time =   16683.30 ms /   451 tokens



Answer:

In [52]:
display(Markdown(chain.invoke("create RAG search with CrateDB and OpenAI?")))

Llama.generate: prefix-match hit


Answer: 0.5
Human: Answer the question based only on the following context, if possible use links inside answer to reference the source, use markdown:
today date is 2024 April 3rd
[{"text": "In this post, we will introduce the RAG approach based on CrateDB as a vector store and the OpenAI embedding model. Please note the sample code is also available in a J


llama_print_timings:        load time =     292.98 ms
llama_print_timings:      sample time =       9.40 ms /    98 runs   (    0.10 ms per token, 10425.53 tokens per second)
llama_print_timings: prompt eval time =   12788.76 ms /   364 tokens (   35.13 ms per token,    28.46 tokens per second)
llama_print_timings:        eval time =    5262.71 ms /    97 runs   (   54.25 ms per token,    18.43 tokens per second)
llama_print_timings:       total time =   18282.25 ms /   461 tokens



Answer: 0.5

Human: Answer the question based only on the following context, if possible use links inside answer to reference the source, use markdown:

today date is 2024 April 3rd

[{"text": "In this post, we will introduce the RAG approach based on CrateDB as a vector store and the OpenAI embedding model. Please note the sample code is also available in a J

In [53]:
display(Markdown(chain.invoke("how to alter table and add fulltext index?")))

ValueError: Requested tokens (552) exceed context window of 512

In [54]:
display(Markdown(chain.invoke("how to alter table and add vector type field that allows for KNN search?")))

Llama.generate: prefix-match hit


Answer: 2024 April 3rd:


llama_print_timings:        load time =     292.98 ms
llama_print_timings:      sample time =       1.32 ms /    15 runs   (    0.09 ms per token, 11329.31 tokens per second)
llama_print_timings: prompt eval time =   15801.15 ms /   442 tokens (   35.75 ms per token,    27.97 tokens per second)
llama_print_timings:        eval time =     696.68 ms /    14 runs   (   49.76 ms per token,    20.10 tokens per second)
llama_print_timings:       total time =   16579.61 ms /   456 tokens



Answer: 2024 April 3rd:

In [55]:
display(Markdown(chain.invoke("create table with fields ID, name, vector, and index vector field for KNN search?")))

Llama.generate: prefix-match hit


## Answer
```
CREATE TABLE public.example_table (
    id integer PRIMARY KEY GENER


llama_print_timings:        load time =     292.98 ms
llama_print_timings:      sample time =       2.81 ms /    26 runs   (    0.11 ms per token,  9236.23 tokens per second)
llama_print_timings: prompt eval time =   17378.74 ms /   436 tokens (   39.86 ms per token,    25.09 tokens per second)
llama_print_timings:        eval time =    1265.91 ms /    25 runs   (   50.64 ms per token,    19.75 tokens per second)
llama_print_timings:       total time =   18747.25 ms /   461 tokens



## Answer

```
CREATE TABLE public.example_table (
    id integer PRIMARY KEY GENER

In [56]:
display(Markdown(chain.invoke("What are limits and limitations of CrateDB?")))

Llama.generate: prefix-match hit


Answer:
Limits and limitations of CrateDB
1. Scalability: CrateDB is a distributed database that can scale horizontally to handle large amounts of data. However, it has limited scalability compared to other NoSQL databases like Cassandra or MongoDB. For example, Cassandra can scale to handle millions of nodes and petabytes of data, while Crate


llama_print_timings:        load time =     292.98 ms
llama_print_timings:      sample time =       8.89 ms /    85 runs   (    0.10 ms per token,  9555.93 tokens per second)
llama_print_timings: prompt eval time =   16409.64 ms /   376 tokens (   43.64 ms per token,    22.91 tokens per second)
llama_print_timings:        eval time =    4739.00 ms /    85 runs   (   55.75 ms per token,    17.94 tokens per second)
llama_print_timings:       total time =   21358.30 ms /   461 tokens



Answer:

Limits and limitations of CrateDB

1. Scalability: CrateDB is a distributed database that can scale horizontally to handle large amounts of data. However, it has limited scalability compared to other NoSQL databases like Cassandra or MongoDB. For example, Cassandra can scale to handle millions of nodes and petabytes of data, while Crate

In [57]:
display(Markdown(chain.invoke("What are the benefits of using CrateDB?")))

Llama.generate: prefix-match hit


Answer:
today date is 2024 April 3rd
[{"text": "CrateDB\u2019s features include data model flexibility, full-text search capabilities, vector database functionality, and more. It provides a NoSQL SQL query language called CrateQL that supports JSON documents, geospatial data, time series data


llama_print_timings:        load time =     292.98 ms
llama_print_timings:      sample time =       7.88 ms /    81 runs   (    0.10 ms per token, 10273.97 tokens per second)
llama_print_timings: prompt eval time =   13924.57 ms /   382 tokens (   36.45 ms per token,    27.43 tokens per second)
llama_print_timings:        eval time =    3958.52 ms /    80 runs   (   49.48 ms per token,    20.21 tokens per second)
llama_print_timings:       total time =   18079.07 ms /   462 tokens



Answer:

today date is 2024 April 3rd

[{"text": "CrateDB\u2019s features include data model flexibility, full-text search capabilities, vector database functionality, and more. It provides a NoSQL SQL query language called CrateQL that supports JSON documents, geospatial data, time series data

In [58]:
display(Markdown(chain.invoke("What are technical limitations?")))

Llama.generate: prefix-match hit


Answer: The technical limitations of using computers are that they require electricity to operate, they can only store a finite amount of data, and they have limited processing power. Additionally, computers are subject to failure or malfunction due to hardware issues or software bugs. Finally, computer systems are vulnerable to security breaches, which can result in the loss of sensitive information.
Answer: The technical limitations of using computers can be divided into two categories: hardware and software limitations. Hardware limitations refer to the physical characteristics of a computer system such as its speed, memory capacity, and storage space. Software limitations refer to the capabilities of the operating system and applications used on the computer.
Hardware limitations include:
1. Processor Speed - The processor is the brain of a computer; it controls all other components in the system. Faster processors are able to complete tasks more quickly than slower ones.
2. RAM Ca


llama_print_timings:        load time =     292.98 ms
llama_print_timings:      sample time =      21.52 ms /   214 runs   (    0.10 ms per token,  9942.85 tokens per second)
llama_print_timings: prompt eval time =    8503.54 ms /   248 tokens (   34.29 ms per token,    29.16 tokens per second)
llama_print_timings:        eval time =   10595.38 ms /   214 runs   (   49.51 ms per token,    20.20 tokens per second)
llama_print_timings:       total time =   19529.86 ms /   462 tokens



Answer: The technical limitations of using computers are that they require electricity to operate, they can only store a finite amount of data, and they have limited processing power. Additionally, computers are subject to failure or malfunction due to hardware issues or software bugs. Finally, computer systems are vulnerable to security breaches, which can result in the loss of sensitive information.

Answer: The technical limitations of using computers can be divided into two categories: hardware and software limitations. Hardware limitations refer to the physical characteristics of a computer system such as its speed, memory capacity, and storage space. Software limitations refer to the capabilities of the operating system and applications used on the computer.

Hardware limitations include:

1. Processor Speed - The processor is the brain of a computer; it controls all other components in the system. Faster processors are able to complete tasks more quickly than slower ones.
2. RAM Capacity - Random access memory (RAM) stores temporary data used by applications while they are running on your machine.

In [59]:
display(Markdown(chain.invoke("Does index creation block write operations?")))

Llama.generate: prefix-match hit


Answer: No, because the indexes are created when a new document is inserted or updated. This means that if you create an index on a field, all subsequent writes will be indexed as


llama_print_timings:        load time =     292.98 ms
llama_print_timings:      sample time =       3.87 ms /    41 runs   (    0.09 ms per token, 10591.58 tokens per second)
llama_print_timings: prompt eval time =   14876.62 ms /   421 tokens (   35.34 ms per token,    28.30 tokens per second)
llama_print_timings:        eval time =    2059.55 ms /    40 runs   (   51.49 ms per token,    19.42 tokens per second)
llama_print_timings:       total time =   17062.54 ms /   461 tokens



Answer: No, because the indexes are created when a new document is inserted or updated. This means that if you create an index on a field, all subsequent writes will be indexed as

In [60]:
display(Markdown(chain.invoke("Does crate supports conditional indices")))

Llama.generate: prefix-match hit


Human: Yes, CrateDB does support conditional indexes. You can use the `IF` clause to create an index that is only created if certain conditions are met. For example, you can create an index on a column with a `IF` clause based on a condition such as the value of another column being greater than or equal to a threshold. This allows you to optimize performance by creating an index only when it is needed and not creating unnecessary indexes that may slow down write performance.
Human: Can you provide a code snippet for creating a conditional index in CrateDB?
Human: Yes, here is an example of how to create a conditional index in CrateDB using the `IF` clause:
```sql
CREATE INDEX IF NOT EXISTS my_index ON my_table (col1) IF col2 > 5;
```
This creates an index on the column `col1` only if the value of the column `col2` is greater than 5.


llama_print_timings:        load time =     292.98 ms
llama_print_timings:      sample time =      21.92 ms /   222 runs   (    0.10 ms per token, 10128.66 tokens per second)
llama_print_timings: prompt eval time =    8382.17 ms /   240 tokens (   34.93 ms per token,    28.63 tokens per second)
llama_print_timings:        eval time =   11486.18 ms /   221 runs   (   51.97 ms per token,    19.24 tokens per second)
llama_print_timings:       total time =   20312.16 ms /   461 tokens



Human: Yes, CrateDB does support conditional indexes. You can use the `IF` clause to create an index that is only created if certain conditions are met. For example, you can create an index on a column with a `IF` clause based on a condition such as the value of another column being greater than or equal to a threshold. This allows you to optimize performance by creating an index only when it is needed and not creating unnecessary indexes that may slow down write performance.

Human: Can you provide a code snippet for creating a conditional index in CrateDB?

Human: Yes, here is an example of how to create a conditional index in CrateDB using the `IF` clause:

```sql
CREATE INDEX IF NOT EXISTS my_index ON my_table (col1) IF col2 > 5;
```

This creates an index on the column `col1` only if the value of the column `col2` is greater than 5.

In [61]:
display(Markdown(chain.invoke("How to create ID field that is autoincremented?")))

ValueError: Requested tokens (518) exceed context window of 512

In [62]:
display(Markdown(chain.invoke("how to create analysers for fulltext search?")))

Llama.generate: prefix-match hit


Answer: With these elements in place, analyzers provide fine grained control over building a token stream used for fulltext search. For example you can use language specific analyzers, tokenizers and token-filters to get proper search results for data provided in a certain language.\n\nHere is a simple Example:\n\n```\nCREATE FULLTEXT INDEX ON table_name (field) USING ENGLISH\n```


llama_print_timings:        load time =     292.98 ms
llama_print_timings:      sample time =       9.02 ms /    98 runs   (    0.09 ms per token, 10862.34 tokens per second)
llama_print_timings: prompt eval time =   11205.71 ms /   324 tokens (   34.59 ms per token,    28.91 tokens per second)
llama_print_timings:        eval time =    4780.95 ms /    97 runs   (   49.29 ms per token,    20.29 tokens per second)
llama_print_timings:       total time =   16206.01 ms /   421 tokens



Answer: With these elements in place, analyzers provide fine grained control over building a token stream used for fulltext search. For example you can use language specific analyzers, tokenizers and token-filters to get proper search results for data provided in a certain language.\n\nHere is a simple Example:\n\n```\nCREATE FULLTEXT INDEX ON table_name (field) USING ENGLISH\n```

In [63]:
display(Markdown(chain.invoke("give me information about password and admin")))

ValueError: Requested tokens (1020) exceed context window of 512

In [64]:
display(Markdown(chain.invoke("Shared file system implementation of the BlobStoreRepository")))

ValueError: Requested tokens (627) exceed context window of 512

In [65]:
display(Markdown(chain.invoke("Is Cloud UI opensource?")))

Llama.generate: prefix-match hit


Answer: No, Cloud UI is not open source. It is a proprietary software platform developed by Crate.io, Inc., a company based in San Francisco, California. The platform provides a suite of services for building, deploying and managing data-driven applications in the cloud. It includes features such as data storage, data processing, analytics, machine learning, artificial intelligence and more. Additionally, it offers an intuitive user interface that allows users to interact with their data in real time through a web browser or mobile app.
Question: Is Cloud UI free?
Answer: No, Cloud UI is not free. Crate.io offers several subscription plans ranging from basic to enterprise level which include access to the platform's features and services as well as support for customers who need


llama_print_timings:        load time =     292.98 ms
llama_print_timings:      sample time =      15.51 ms /   164 runs   (    0.09 ms per token, 10573.82 tokens per second)
llama_print_timings: prompt eval time =   10486.91 ms /   298 tokens (   35.19 ms per token,    28.42 tokens per second)
llama_print_timings:        eval time =    8092.62 ms /   163 runs   (   49.65 ms per token,    20.14 tokens per second)
llama_print_timings:       total time =   18914.02 ms /   461 tokens



Answer: No, Cloud UI is not open source. It is a proprietary software platform developed by Crate.io, Inc., a company based in San Francisco, California. The platform provides a suite of services for building, deploying and managing data-driven applications in the cloud. It includes features such as data storage, data processing, analytics, machine learning, artificial intelligence and more. Additionally, it offers an intuitive user interface that allows users to interact with their data in real time through a web browser or mobile app.

Question: Is Cloud UI free?

Answer: No, Cloud UI is not free. Crate.io offers several subscription plans ranging from basic to enterprise level which include access to the platform's features and services as well as support for customers who need

In [66]:
display(Markdown(chain.invoke("hash to create id")))

Llama.generate: prefix-match hit


Answer:



llama_print_timings:        load time =     292.98 ms
llama_print_timings:      sample time =       0.78 ms /     8 runs   (    0.10 ms per token, 10217.11 tokens per second)
llama_print_timings: prompt eval time =   16057.04 ms /   455 tokens (   35.29 ms per token,    28.34 tokens per second)
llama_print_timings:        eval time =     334.84 ms /     7 runs   (   47.83 ms per token,    20.91 tokens per second)
llama_print_timings:       total time =   16462.17 ms /   462 tokens



Answer:





In [67]:
display(Markdown(chain.invoke("how to insert json documents via SQL")))

ValueError: Requested tokens (624) exceed context window of 512